# Lab 10.3: Parental Investment Simulator
## Chapter 10: The Reproduction Revolution

### 🎯 Learning Objectives
- Calculate parental investment costs (time, energy, risk)
- Analyze investment benefits for offspring survival
- Compare uniparental, biparental, and cooperative care
- Evaluate trade-offs between current and future reproduction
- Apply investment theory to extreme cases

### 📖 Connection to Chapter 10
This lab integrates **Section 10.4: Parental Care**:
- Parental investment spectrum (none to extreme)
- Emperor penguin 64-day fast
- Seahorse male pregnancy
- Poison dart frog intensive care
- Cooperative breeding (scrub-jays)
- Trade-offs: current vs future reproduction

### 👶 The Question
**Why do male emperor penguins fast 64 days while seahorses get pregnant?**  
What are the costs and benefits? Let's calculate!

In [ ]:
# === GOOGLE COLAB SETUP ===
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✓ Widgets enabled")
except:
    print("✓ Running outside Colab")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import *
from IPython.display import display, clear_output
from datetime import datetime

print("✓ Ready!")

## Part 1: Investment Theory

### The Framework
**Parental investment** = any investment that:
1. Increases offspring survival
2. Decreases parent's future reproduction

### Costs to Parent
**Time**: Days caring → delayed next reproduction

**Energy**: Calories spent → reduced parent survival

$$E_{total} = E_{daily} \times days$$

**Risk**: Mortality from care → may not reproduce again

$$P_{survive} = 1 - risk \times \frac{days}{365}$$

### Benefits to Offspring
**Survival boost**:

$$S = S_{base} + boost \times (1 - e^{-0.01 \times days})$$

### The Trade-off
**Lifetime Fitness** = Current + Future

$$F = (n_1 \times S_1 \times P) + (n_2 \times S_2 \times P)$$

Where n = offspring, S = survival, P = parent survival

In [ ]:
# Functions
def calc_offspring_survival(care_days, base_surv, boost):
    return base_surv + boost * (1 - np.exp(-0.01 * care_days))

def calc_parent_survival(care_days, risk):
    return max(0.01, 1 - risk * (care_days / 365))

def calc_fitness(n_offspring, care_days, base_surv, boost, risk):
    s = calc_offspring_survival(care_days, base_surv, boost)
    p = calc_parent_survival(care_days, risk)
    return n_offspring * s * p

# Test
print("EXAMPLES:")
print("="*60)
# Penguin
s = calc_offspring_survival(64, 0.10, 0.50)
p = calc_parent_survival(64, 0.15)
f = 1 * s * p
print(f"Emperor Penguin (64 days):")
print(f"  Offspring survival: {s:.1%}")
print(f"  Parent survival: {p:.1%}")
print(f"  Fitness: {f:.2f}\n")

# Cod
s2 = calc_offspring_survival(0, 0.00001, 0)
print(f"Cod (no care):")
print(f"  Offspring survival: {s2:.4%}")
print(f"  But 9M eggs → {9000000*s2:.0f} survivors")
print("\n✓ Ready!")

## Part 2: Species Database

In [ ]:
species_care = {
    'Emperor Penguin': {
        'care_days': 64, 'energy_kcal': 120000, 'risk': 0.15,
        'base_survival': 0.10, 'boost': 0.50, 'offspring': 1,
        'type': 'Extreme male', 'parent': 'Male',
        'desc': 'Male fasts 64 days incubating egg in Antarctic winter! '
                'Loses 45% body weight. Female hunts at sea.'
    },
    'Seahorse': {
        'care_days': 14, 'energy_kcal': 5000, 'risk': 0.05,
        'base_survival': 0.05, 'boost': 0.70, 'offspring': 100,
        'type': 'Male pregnancy', 'parent': 'Male',
        'desc': 'Male pregnancy! Eggs develop in brood pouch. Male provides '
                'nutrients and oxygen. Gives birth to live young.'
    },
    'Poison Dart Frog': {
        'care_days': 60, 'energy_kcal': 200, 'risk': 0.10,
        'base_survival': 0.05, 'boost': 0.65, 'offspring': 6,
        'type': 'Intensive biparental', 'parent': 'Both',
        'desc': 'Both parents care! Transport tadpoles individually to water. '
                'Mother feeds each tadpole unfertilized eggs weekly.'
    },
    'Prairie Vole': {
        'care_days': 21, 'energy_kcal': 3000, 'risk': 0.08,
        'base_survival': 0.10, 'boost': 0.30, 'offspring': 4,
        'type': 'Biparental', 'parent': 'Both',
        'desc': 'Unusual mammal with biparental care. Monogamous pairs. '
                'Both parents provision nest and protect young.'
    },
    'Florida Scrub-Jay': {
        'care_days': 80, 'energy_kcal': 8000, 'risk': 0.12,
        'base_survival': 0.15, 'boost': 0.45, 'offspring': 3,
        'type': 'Cooperative', 'parent': 'Both + helpers',
        'helpers': 2.5,
        'desc': 'Cooperative breeding! 2-4 helpers (usually offspring from '
                'previous years) assist parents. Increases survival 30%.'
    },
    'Red Deer': {
        'care_days': 240, 'energy_kcal': 180000, 'risk': 0.10,
        'base_survival': 0.15, 'boost': 0.70, 'offspring': 1,
        'type': 'Maternal', 'parent': 'Female',
        'desc': 'Mother-only care for 8 months. Milk production costs huge '
                'energy. Investment varies with maternal condition.'
    },
    'Cod': {
        'care_days': 0, 'energy_kcal': 0, 'risk': 0.00,
        'base_survival': 0.00001, 'boost': 0.00, 'offspring': 9000000,
        'type': 'No care', 'parent': 'None',
        'desc': 'Baseline: no parental care. Broadcast spawning. '
                '9 million eggs but only ~90 survive. r-selection extreme.'
    },
    'Elephant': {
        'care_days': 3650, 'energy_kcal': 5000000, 'risk': 0.05,
        'base_survival': 0.05, 'boost': 0.80, 'offspring': 1,
        'type': 'Extreme maternal + social', 'parent': 'Female + herd',
        'desc': 'Most extreme care! 10 years of maternal investment. '
                'Social learning critical. One calf every 4-5 years.'
    }
}

# Calculate for all
for name in species_care:
    sp = species_care[name]
    sp['survival'] = calc_offspring_survival(sp['care_days'], sp['base_survival'], sp['boost'])
    sp['parent_survive'] = calc_parent_survival(sp['care_days'], sp['risk'])
    sp['fitness'] = sp['offspring'] * sp['survival'] * sp['parent_survive']

print("PARENTAL CARE DATABASE")
print("="*80)
print(f"{'Species':<20}{'Days':<8}{'Type':<20}{'Offspring':<12}{'Fitness'}")
print("="*80)
for name in sorted(species_care.keys(), key=lambda x: species_care[x]['care_days']):
    sp = species_care[name]
    print(f"{name:<20}{sp['care_days']:<8}{sp['type']:<20}{sp['offspring']:<12,}{sp['fitness']:.1f}")
print("="*80)
print("✓ Database ready!")

## Part 3: Investment Calculator

In [ ]:
def analyze_investment(species_name):
    sp = species_care[species_name]
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Investment Costs', 'Offspring Benefits',
                       'Cost-Benefit Analysis', 'Parent Risk'),
        specs=[[{'type': 'bar'}, {'type': 'indicator'}],
               [{'type': 'scatter'}, {'type': 'indicator'}]]
    )
    
    # 1. Costs
    costs = ['Time\n(days)', 'Energy\n(kcal)', 'Risk\n(%)']
    values = [sp['care_days'], sp['energy_kcal'], sp['risk']*100]
    fig.add_trace(go.Bar(
        x=costs, y=values,
        marker_color=['#E74C3C', '#F39C12', '#9B59B6'],
        text=[f"{v:,.0f}" for v in values],
        textposition='outside'
    ), row=1, col=1)
    
    # 2. Offspring survival gauge
    fig.add_trace(go.Indicator(
        mode="gauge+number+delta",
        value=sp['survival']*100,
        title={'text': "Survival %"},
        delta={'reference': sp['base_survival']*100},
        gauge={'axis': {'range': [0, 100]},
               'bar': {'color': '#2ECC71'}}
    ), row=1, col=2)
    
    # 3. Cost-benefit scatter
    for sp_name, sp_data in species_care.items():
        is_current = (sp_name == species_name)
        fig.add_trace(go.Scatter(
            x=[sp_data['care_days']], y=[sp_data['survival']],
            mode='markers+text' if is_current else 'markers',
            marker=dict(size=15 if is_current else 8,
                       color='#E74C3C' if is_current else '#95A5A6'),
            text=[sp_name] if is_current else None,
            textposition='top center',
            showlegend=False,
            hovertemplate=f"{sp_name}<br>Care: {sp_data['care_days']}d<br>Survival: {sp_data['survival']:.1%}<extra></extra>"
        ), row=2, col=1)
    
    # 4. Parent survival
    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=sp['parent_survive']*100,
        title={'text': "Parent Survives"},
        gauge={'axis': {'range': [0, 100]},
               'bar': {'color': '#3498DB'}}
    ), row=2, col=2)
    
    fig.update_xaxes(title_text="Care Duration (days)", row=2, col=1)
    fig.update_yaxes(title_text="Offspring Survival", row=2, col=1)
    fig.update_layout(height=700, title_text=f"<b>{species_name}</b>")
    
    # Summary
    print("\n" + "="*70)
    print(f"{species_name.upper()}: {sp['type']}")
    print("="*70)
    print(f"Care provider: {sp['parent']}")
    print(f"\nCOSTS:")
    print(f"  Time: {sp['care_days']} days")
    print(f"  Energy: {sp['energy_kcal']:,} kcal")
    print(f"  Mortality risk: {sp['risk']:.1%}")
    print(f"\nBENEFITS:")
    print(f"  Base survival: {sp['base_survival']:.1%}")
    print(f"  With care: {sp['survival']:.1%} ({sp['survival']/sp['base_survival'] if sp['base_survival']>0 else 0:.1f}× boost)")
    print(f"\nFITNESS:")
    print(f"  {sp['offspring']} offspring × {sp['survival']:.1%} × {sp['parent_survive']:.1%} = {sp['fitness']:.1f}")
    print(f"\n{sp['desc']}")
    print("="*70)
    
    fig.show()

species_dropdown = Dropdown(
    options=sorted(species_care.keys()),
    value='Emperor Penguin',
    description='Species:'
)

display(HTML("<h3>👶 Investment Analyzer</h3>"))
interact(analyze_investment, species_name=species_dropdown);

## Part 4: Care Type Comparison

In [ ]:
def compare_care_types():
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Investment vs Survival', 'Fitness by Care Type')
    )
    
    # 1. Scatter
    for name, sp in species_care.items():
        fig.add_trace(go.Scatter(
            x=[sp['care_days']], y=[sp['survival']],
            mode='markers+text', text=[name],
            textposition='top center',
            marker=dict(size=12), name=name,
            showlegend=False
        ), row=1, col=1)
    
    # 2. Bar by type
    types = {}
    for sp in species_care.values():
        t = sp['parent']
        if t not in types: types[t] = []
        types[t].append(sp['fitness'])
    
    fig.add_trace(go.Bar(
        x=list(types.keys()),
        y=[np.mean(types[t]) for t in types],
        marker_color='#3498DB',
        text=[f"{np.mean(types[t]):.1f}" for t in types],
        textposition='outside'
    ), row=1, col=2)
    
    fig.update_xaxes(title_text="Care Days", row=1, col=1)
    fig.update_yaxes(title_text="Survival", row=1, col=1)
    fig.update_yaxes(title_text="Avg Fitness", row=1, col=2)
    fig.update_layout(height=500, title_text='<b>Care Type Comparison</b>')
    
    print("\nCOMPARISON BY CARE PROVIDER")
    print("="*60)
    for t in types:
        print(f"{t}: {np.mean(types[t]):.1f} avg fitness")
    print("="*60)
    
    fig.show()

btn = Button(description='📊 Compare', button_style='info')
out = Output()
def on_click(b):
    with out:
        clear_output(wait=True)
        compare_care_types()
btn.on_click(on_click)
display(HTML("<h3>📊 Compare All</h3>"))
display(btn, out)

## Part 5: Challenges

### Challenge 1: Emperor Penguin Economics 🐧

**Question**: How does a male penguin survive 64 days without food?

**Given**:
- Male penguin mass: 40 kg
- Fat stores: 15 kg (35% body weight)
- Daily energy need: 1,875 kcal
- Fat energy: 9 kcal/g

**Calculate**:
1. Total energy needed for 64 days?
2. Energy from fat stores?
3. Enough to survive?

<details>
<summary>Solution</summary>

**Total energy needed**:
- 64 days × 1,875 kcal/day = 120,000 kcal

**Energy from fat**:
- 15 kg fat × 1000 g/kg × 9 kcal/g = 135,000 kcal

**Result**: YES! Just barely.
- Has 135,000 kcal, needs 120,000
- Loses ~13.3 kg (33% body weight)
- Down to ~27 kg by end

**Why males?** Females better at ocean hunting. Males have territory. Role specialization!
</details>

In [ ]:
# Challenge 1 workspace
print("CHALLENGE 1: PENGUIN ENERGETICS")
print("="*60)
days = 64
daily_kcal = 1875
fat_kg = 15
fat_kcal_per_g = 9

total_needed = days * daily_kcal
total_stored = fat_kg * 1000 * fat_kcal_per_g
fat_used_kg = total_needed / (1000 * fat_kcal_per_g)

print(f"Energy needed: {total_needed:,} kcal")
print(f"Energy stored: {total_stored:,} kcal")
print(f"Fat used: {fat_used_kg:.1f} kg")
print(f"\nSufficient? {'YES' if total_stored >= total_needed else 'NO'}!")
print(f"Survival margin: {total_stored - total_needed:,} kcal")
print("="*60)

### Challenge 2: Cooperative Breeding 🐦

**Question**: When does helping relatives pay off?

**Hamilton's Rule**: Help when **rb > c**
- r = relatedness (0.5 for siblings, 0.25 for nieces/nephews)
- b = benefit to recipient (extra offspring survive)
- c = cost to helper (lost own reproduction)

**Scrub-Jay Example**:
- Helper (offspring from last year) can:
  - A: Breed alone (20% chance 2 offspring survive)
  - B: Help parents (boost siblings from 40% → 70% survival, 3 siblings)

**Calculate**: Which maximizes fitness?

<details>
<summary>Solution</summary>

**Option A (breed alone)**:
- Direct fitness: 0.20 × 2 = 0.4 offspring

**Option B (help parents)**:
- Siblings without help: 3 × 0.40 = 1.2 survive
- Siblings with help: 3 × 0.70 = 2.1 survive
- Extra survivors: 2.1 - 1.2 = 0.9
- Indirect fitness: 0.5 (relatedness) × 0.9 = 0.45

**Result**: HELP! (0.45 > 0.4)

Helping gives slightly higher fitness through kin selection!
</details>

### Challenge 3: Optimal Care Duration ⏱️

**Design**: You're a bird. How long should you care for chicks?

**Trade-off**:
- More care → higher chick survival
- More care → fewer future broods

**Parameters**:
- Chick base survival: 30%
- Care boost: +5% per 10 days (max 80%)
- Breeding season: 120 days
- Time between broods: care_days + 20 days

**Find**: Optimal care duration to maximize total offspring?

<details>
<summary>Solution</summary>

**Calculate for different durations**:
- 20 days: 80% survival, 2.4 broods → 4.8 offspring
- 30 days: 85% survival, 1.92 broods → 5.1 offspring ✓
- 40 days: 90% survival, 1.5 broods → 4.5 offspring
- 50 days: 80% survival, 1.26 broods → 4.03 offspring

**Optimal**: ~30 days care
- Balance between quality and quantity
- Too short: low survival
- Too long: miss opportunities

Real birds show this pattern!
</details>

## Part 6: Export

In [ ]:
def export_results():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "/content"
    
    data = []
    for name, sp in species_care.items():
        data.append({
            'Species': name, 'Type': sp['type'],
            'Care_Days': sp['care_days'],
            'Energy_kcal': sp['energy_kcal'],
            'Risk': sp['risk'],
            'Survival': sp['survival'],
            'Fitness': sp['fitness']
        })
    
    df = pd.DataFrame(data)
    csv_file = f"{output_dir}/lab_10_2_investment_{timestamp}.csv"
    df.to_csv(csv_file, index=False)
    print(f"✓ Saved: {csv_file}")
    print("\nTo download: Click 📁 → /content → right-click → Download")

btn = Button(description='📥 Export', button_style='success', icon='download')
btn.on_click(lambda b: export_results())
display(HTML("<h3>📤 Export</h3>"))
display(btn)

## Summary

### Key Insights

✅ **Investment costs**: Time, energy, risk  
✅ **Benefits**: Offspring survival boost  
✅ **Trade-offs**: Current vs future reproduction  
✅ **Extremes**: Penguin 64-day fast, Elephant 10-year care  
✅ **Cooperation**: Helpers when rb > c  

### Amazing Facts

- Emperor penguin loses 45% body weight!
- Seahorse males get pregnant
- Poison dart frog feeds each tadpole individually
- Scrub-jay helpers boost survival 30%
- Elephants care for 10 years!

### The Pattern

**No universal best strategy**:
- No care works (cod: 9M eggs)
- Extreme care works (elephant: 1 calf)
- Environment + lifespan determine optimal

**Congratulations!** 🎉